In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
sys.path.append('/mnt/sdb1/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [5]:
%matplotlib
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as pl
import pickle as pkl
import fnmatch,os
import itertools
import statsmodels.formula.api as sm
from scipy.stats import pearsonr,ttest_ind,ranksums,linregress,ks_2samp,entropy,mannwhitneyu
import pandas as pd
import seaborn as sns
from signale.tools import locate
import statsmodels.api as sm
from matplotlib.gridspec import GridSpec
from scipy.cluster.hierarchy import fcluster,linkage,dendrogram
from scipy.io import savemat,loadmat
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

Using matplotlib backend: Qt5Agg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/mnt/sdb1/softwares/anaconda2/envs/caiman/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [76]:
sns.set_context("paper",font_scale=1.6)
colors = ["windows blue", "amber", "greyish", "faded green", "dusty purple"]
leftRight = sns.color_palette(['#f9ba32','#426e86'])
seabC = sns.xkcd_palette(colors)
seabP = sns.color_palette("Paired")
flatui = sns.color_palette([  "#34495e","#e74c3c", "#2ecc71","#9b59b6", "#3498db", "#95a5a6"])
#sns.palplot(flatui)
sns.set_style('ticks')
kr = sns.color_palette(['k','r'])
#############################################################
clr4 = ["#cb535a","#5ba952","#c97d3d","#9a993e"]
cl3 = ["#cb6a49","#a46cb7","#7aa457"]
clrs = ["#e55982","#41c563","#874ac4","#77ba3f","#b171ed","#aab632","#4e6ddc","#408f2c","#de5fcd","#80bc6a",
        "#a63798","#5bc79f","#dd378e","#508d4e","#c174d6","#cca238","#754da1","#e18e31","#9488e0","#dd6a35","#4fbddc",
        "#c23d26","#5d92d3","#d33f4f","#379680","#da63a9","#416831","#e38edb","#7a7529","#5163a5","#b0b166","#b03063",
        "#da9c6b","#a96cae","#995e2b","#bea0df","#d77768","#97527f","#e08aa5","#9f4d5a"]
colors = ['#FF0000', '#0000FF', '#008000', '#00FFFF', '#FF00FF', '#EE82EE',
        '#808000', '#800080', '#FF6347', '#FFFF00', '#9ACD32', '#4B0082',
        '#FFFACD', '#C0C0C0', '#A0522D', '#FA8072', '#FFEFD5', '#E6E6FA',
        '#F1FAC1', '#C5C5C5', '#A152ED', '#FADD72', '#F0EFD0', '#EEE6FF',
        '#01FAC1', '#F5F5F5', '#A152FF', '#FAFD72', '#F0EFDF', '#EEEFFF',
        '#F1FA99', '#C9C9C9', '#A152DD', '#FA5572', '#FFFFD0', '#EDD6FF']

In [127]:
lesTh  =  ['/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat3656-D1.db-theta-vis.lsn',
            '/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat3754-D2.db-theta-vis.lsn',
            '/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat3837-D1.db-theta-vis.lsn',
            '/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat3838-D2.db-theta-vis.lsn',
            '/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat3903-D1.db-theta-vis.lsn',
            '/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat405-D1.db-100ms-vis.lsn']
cntTh  = ['/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat3906-D2.db-theta-vis.cnt',
          '/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat3839-D4.db-theta-vis.cnt',  
          '/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat3661-D3.db-theta-vis.cnt',
          '/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat614-D1.db-theta-vis.cnt',
          '/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA_REV/Rat616-D1.db-theta-vis.cnt']
##############################################################################
###Theta
lesThPre = np.array([])
lesThImm = np.array([])
lesThPost = np.array([])
for item in lesTh:
    lesThPre  = np.append(lesThPre,pkl.load(open(item,'rb'))[0])
    lesThImm  = np.append(lesThImm,pkl.load(open(item,'rb'))[1])
    lesThPost = np.append(lesThPost,pkl.load(open(item,'rb'))[2])
##############################################################################
    cntThPre = np.array([])
cntThImm = np.array([])
cntThPost = np.array([])
for item in cntTh:
    cntThPre  = np.append(cntThPre,pkl.load(open(item,'rb'))[0])
    cntThImm  = np.append(cntThImm,pkl.load(open(item,'rb'))[1])
    cntThPost = np.append(cntThPost,pkl.load(open(item,'rb'))[2])

##############################################################################

In [128]:
pl.scatter(lesThPre,lesThPost)
pl.scatter(cntThPre,cntThPost)

In [129]:
ThdistL = np.sqrt(np.square(lesThPost)+np.square(lesThPre))
ThdistC = np.sqrt(np.square(cntThPost)+np.square(cntThPre))
outerC = np.logical_or(cntThPost>1.18,cntThPre>1.13)
outerL = np.logical_or(lesThPost>1.18,lesThPre>1.13)
######################
ddf = pd.DataFrame([],columns=['d','range','group'])
for ii,distances in enumerate([ThdistC,ThdistL]):
    gr = ['control','lesioned']
    for d in distances:
        if d<1.5:
            ddf = ddf.append(pd.DataFrame([[d,'short',gr[ii]]],columns=['d','range','group']),ignore_index=True)
        elif d<1.9:
            ddf = ddf.append(pd.DataFrame([[d,'medium',gr[ii]]],columns=['d','range','group']),ignore_index=True)
        else:
            ddf = ddf.append(pd.DataFrame([[d,'long',gr[ii]]],columns=['d','range','group']),ignore_index=True)
##############################
ThangL = np.arctan(lesThPost[outerL]/lesThPre[outerL])/np.pi*180
ThangC = np.arctan(cntThPost[outerC]/cntThPre[outerC])/np.pi*180
aaf = pd.DataFrame([],columns=['d','region','group'])
for ii,angles in enumerate([ThangC,ThangL]):
    gr = ['control','lesioned']
    for a in angles:
        if a<43:
            aaf = aaf.append(pd.DataFrame([[a,'lower',gr[ii]]],columns=['d','region','group']),ignore_index=True)
        elif a<47:
            aaf = aaf.append(pd.DataFrame([[a,'middle',gr[ii]]],columns=['d','region','group']),ignore_index=True)
        else:
            aaf = aaf.append(pd.DataFrame([[a,'upper',gr[ii]]],columns=['d','region','group']),ignore_index=True)
##############################
ThprcL = 1e2*(lesThPost-lesThPre)/(lesThPost+lesThPre)
ThprcC = 1e2*(cntThPost-cntThPre)/(cntThPost+cntThPre)
ppf = pd.DataFrame([],columns=['p','range','group'])
for ii,percents in enumerate([ThprcC,ThprcL]):
    gr = ['control','lesioned']
    for p in percents:
        if p<-50:
            ppf = ppf.append(pd.DataFrame([[p,'least',gr[ii]]],columns=['p','range','group']),ignore_index=True)
        elif p<0:
            ppf = ppf.append(pd.DataFrame([[p,'neg',gr[ii]]],columns=['p','range','group']),ignore_index=True)
        elif p<5:
            ppf = ppf.append(pd.DataFrame([[p,'pos',gr[ii]]],columns=['p','range','group']),ignore_index=True)
        else:
            ppf = ppf.append(pd.DataFrame([[p,'most',gr[ii]]],columns=['p','range','group']),ignore_index=True)
##############################
immThprcL = 1e2*(lesThImm-lesThPre)/(lesThImm+lesThPre)
immThprcC = 1e2*(cntThImm-cntThPre)/(cntThImm+cntThPre)
immppf = pd.DataFrame([],columns=['p','range','group'])
for ii,percents in enumerate([immThprcC,immThprcL]):
    gr = ['control','lesioned']
    for p in percents:
        if p<-50:
            immppf = immppf.append(pd.DataFrame([[p,'least',gr[ii]]],columns=['p','range','group']),ignore_index=True)
        elif p<0:
            immppf = immppf.append(pd.DataFrame([[p,'neg',gr[ii]]],columns=['p','range','group']),ignore_index=True)
        elif p<5:
            immppf = immppf.append(pd.DataFrame([[p,'pos',gr[ii]]],columns=['p','range','group']),ignore_index=True)
        else:
            immppf = immppf.append(pd.DataFrame([[p,'most',gr[ii]]],columns=['p','range','group']),ignore_index=True)

In [130]:
f,(ax11,ax111) = pl.subplots(2,1,figsize=[6,9])
cDkde = sm.nonparametric.KDEUnivariate(cntThPost[outerC])#ddf[ddf.group=='control'].d)
lDkde = sm.nonparametric.KDEUnivariate(lesThPost[outerL])#ddf[ddf.group=='lesioned'].d)
cDkde_imm = sm.nonparametric.KDEUnivariate(cntThImm[outerC])#ddf[ddf.group=='control'].d)
lDkde_imm = sm.nonparametric.KDEUnivariate(lesThImm[outerL])#
cDkde.fit()
bw = cDkde.bw
lDkde.fit(bw=bw)
cDkde_imm.fit(bw=bw)
lDkde_imm.fit(bw=bw)
#ax10.plot(cDkde.support,cDkde.cdf,'k')
#ax10.plot(lDkde.support,lDkde.cdf,'r')
#sns.boxplot(x='range',y='d',data=ddf,hue='group',order=['short','medium','long'],linewidth=1,ax=ax10,palette=kr)
#ax10.set_ylim([0.9,3.1])
#ax10.set_xticks([1,3,5])
#ax10.set_xlabel(r'$A_{POST}$')
ax11.set_ylabel(' Cumulative probability')
#ax10.set_xticklabels(['low','mid','high'])
#ax10.set_yticks([1,2,3])
#ax10.legend(['Contol','Lesioned'])
#ax10.set_ylim(-0.05,1.05)
#ax10.set_yticks([0,0.5,1])
#ax10.xaxis.tick_bottom()
#ax10.yaxis.tick_left()
###############
cAkde = sm.nonparametric.KDEUnivariate(aaf[aaf.group=='control'].d)
lAkde = sm.nonparametric.KDEUnivariate(aaf[aaf.group=='lesioned'].d)
cAkde.fit()
lAkde.fit()
#ax11.plot(cAkde.support,cAkde.cdf,'k')
#ax11.plot(lAkde.support,lAkde.cdf,'r')
#sns.boxplot(x='region',y='d',data=aaf,hue='group',order=['lower','middle','upper'],linewidth=1,ax=ax11,palette=kr)
#ax11.set_ylim([-0.05,1.05])
#ax11.legend('')
#ax11.set_xticks([45,55,65])
#ax11.set_yticks([0,0.5,1])
#ax11.set_xticklabels([1.0,1.4,2.1])
#ax11.set_xticklabels([r'$\vee$',r'$\times$',r'$\wedge$'],fontsize=20)
#ax11.set_xlabel('Ratio')
#ax11.set_ylabel('')
#ax11.xaxis.tick_bottom()
#ax11.yaxis.tick_left()
##############
cCkde = sm.nonparametric.KDEUnivariate(ppf[ppf.group=='control'].p)
lCkde = sm.nonparametric.KDEUnivariate(ppf[ppf.group=='lesioned'].p)
immcCkde = sm.nonparametric.KDEUnivariate(immppf[immppf.group=='control'].p)
immlCkde = sm.nonparametric.KDEUnivariate(immppf[immppf.group=='lesioned'].p)
cCkde.fit()
Cbw = cCkde.bw
lCkde.fit(bw =Cbw)
immlCkde.fit(bw =Cbw)
immcCkde.fit(bw =Cbw)
ax11.plot(cCkde.support,cCkde.cdf,'k')
ax11.plot(lCkde.support,lCkde.cdf,'r')
ax111.plot(immcCkde.support,immcCkde.cdf,'k')
ax111.plot(immlCkde.support,immlCkde.cdf,'r')
ax11.axvline(0,linewidth=1,color='b',alpha=0.5)
ax111.axvline(0,linewidth=1,color='b',alpha=0.5)
#sns.boxplot(x='range',y='p',data=ppf,hue='group',order=['least','neg','pos','most'],linewidth=1,ax=ax12,palette=kr)
ax11.set_ylim([-0.05,1.05])
#ax12.set_xlim([0.5,3.5])
#ax12.set_xticklabels([r'$\nearrow$',r'$\searrow$','-',r'$\nearrow$'],fontsize=20)
ax11.set_xticks([-20,0,20,40])
#ax111.set_xticks([-20,0,20,40])
ax11.set_yticks([0,0.5,1])
ax111.set_yticks([])
ax11.set_xlabel('Relative change (%)')
#ax12.set_ylabel('%')
ax11.legend('')
ax11.xaxis.tick_bottom()
ax11.yaxis.tick_left()
##############
##############
ff,(ax9,ax91) = pl.subplots(1,2,figsize=[10,6])
contourSet = pkl.load(open('/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/shuffledGammaContours.pkl','rb'))['contours']
selContourIdx = pkl.load(open('/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/shuffledGammaContours.pkl','rb'))['selectedContours']
for idx in selContourIdx[:]:
    #pl.scatter(idx+1,cdfArray[idx+1],c='r',s=50)
    pth = contourSet[idx+1].get_paths()[0]
    ax9.plot(pth.vertices[:,0],pth.vertices[:,1],c=seabC[0],lw=1)
    ax91.plot(pth.vertices[:,0],pth.vertices[:,1],c=seabC[0],lw=1)
ax9.plot(lesThPre,lesThPost,'r.')
ax9.plot(cntThPre,cntThPost,'k.')
ax91.plot(lesThPre,lesThImm,'r.')
ax91.plot(cntThPre,cntThImm,'k.')
ax9.set_xticks([0,1,2,3])
ax9.set_yticks([0,1,2,3])
ax91.set_xticks([0,1,2,3])
ax91.set_yticks([0,1,2,3])
ax9.set_ylim([0.8,3.2])
ax9.set_xlim([0.8,4.2])
ax91.set_ylim([0.8,3.2])
ax91.set_xlim([0.8,3.2])
ax9.yaxis.tick_left()
ax9.xaxis.tick_bottom()
ax91.yaxis.tick_left()
ax91.xaxis.tick_bottom()
ax9.set_ylabel(r'$Mean \; reactivation_{POST}}$')
ax9.set_xlabel(r'$Mean \; reactivation_{PRE}}$')
ax91.set_ylabel(r'$Mean \; reactivation_{RUN}}$')
ax91.set_xlabel(r'$Mean \; reactivation_{PRE}}$')

#################################
axins = inset_axes(ax9,width="50%",height="50%",loc=4)
axins9 = inset_axes(ax91,width="50%",height="50%",loc=4)
axins.plot(lesThPre,lesThPost,'r.')
axins.plot(cntThPre,cntThPost,'k.')
axins9.plot(lesThPre,lesThImm,'r.')
axins9.plot(cntThPre,cntThImm,'k.')
for idx in selContourIdx:
    pth = contourSet[idx+1].get_paths()[0]
    axins.plot(pth.vertices[:,0],pth.vertices[:,1],c=seabC[0])
    axins9.plot(pth.vertices[:,0],pth.vertices[:,1],c=seabC[0])
axins.set_xlim(0.82,1.22)
axins.set_ylim(0.82,1.22)
axins9.set_xlim(0.82,1.22)
axins9.set_ylim(0.82,1.22)
pl.setp(axins.get_yticklabels(), visible=False)
pl.setp(axins.get_xticklabels(), visible=False)
pl.setp(axins.get_yticklines(),visible=False)
pl.setp(axins.get_xticklines(),visible=False);
pl.setp(axins9.get_yticklabels(), visible=False)
pl.setp(axins9.get_xticklabels(), visible=False)
pl.setp(axins9.get_yticklines(),visible=False)
pl.setp(axins9.get_xticklines(),visible=False);

In [131]:
slopeDB = pd.DataFrame([],columns=['slope','group'])
for item in cntTh:
    gmm = pkl.load(open(item,'rb'))
    Lregress_imm = sm.OLS( gmm[1],gmm[0] ).fit()
    Lregress = sm.OLS( gmm[2],gmm[0] ).fit()
    slopeDB = slopeDB.append(pd.DataFrame([[Lregress_imm.params[0],'control_im']],columns=['slope','group']),ignore_index=True)
    slopeDB = slopeDB.append(pd.DataFrame([[Lregress.params[0],'control_po']],columns=['slope','group']),ignore_index=True)
for item in lesTh:
    gmm = pkl.load(open(item,'rb'))
    Lregress_imm = sm.OLS( gmm[1],gmm[0] ).fit()
    Lregress = sm.OLS( gmm[2],gmm[0] ).fit()
    slopeDB = slopeDB.append(pd.DataFrame([[Lregress_imm.params[0],'lesioned_im']],columns=['slope','group']),ignore_index=True)
    slopeDB = slopeDB.append(pd.DataFrame([[Lregress.params[0],'lesioned_po']],columns=['slope','group']),ignore_index=True)

In [125]:
slopeDB_orig

,slope,group
0,1.524710,control_im
1,1.281912,control_po
2,1.456089,control_im
3,1.118365,control_po
4,1.448889,control_im
5,1.233828,control_po
6,1.124006,control_im
7,1.008750,control_po
8,1.068186,control_im
9,1.729700,control_po


In [132]:
slopeDB_orig = pd.read_pickle('/mnt/sdb1/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Slopes.db')
ff,ax12 = pl.subplots(1,1,figsize=[6,5])
#sns.swarmplot(x='group',y='slope',data=slopeDB,palette=kr,size=10,order=['control_im','lesioned_im','control_po','lesioned_po'],ax=ax12)
sns.boxplot(x='group',y='slope',data=slopeDB,palette=kr,ax=ax12,linewidth=2.5,fliersize=1,order=['control_im','lesioned_im','control_po','lesioned_po'])
for i,artist in enumerate(ax12.artists):
    # Set the linecolor on the artist to the facecolor, and set the facecolor to None
    col = artist.get_facecolor()
    artist.set_edgecolor(col)
    artist.set_facecolor('None')

    # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
    # Loop over them here, and use the same colour as above
    for j in range(i*6,i*6+6):
        line = ax12.lines[j]
        line.set_color(col)
        line.set_mfc(col)
        line.set_mec(col)
ax12.xaxis.tick_bottom()
#ax12.set_xticklabels(['Ctr.','Les.'])
ax12.yaxis.tick_left()
ax12.set_yticks([1.0,1.2,1.4,1.6,1.8])
#ax12.set_ylim(0.85,1.85)
ax12.set_ylabel('Regression slope')
ax12.set_xlabel('')

Text(0.5,0,u'')

In [66]:
ax9.cla()
axins.cla()
#ax10.cla()
ax11.cla()
ax12.cla()

In [174]:
ax11.axvline(0,linewidth=1,alpha=0.5)

In [91]:
ffx,ax121 = pl.subplots(1,1)
combList = list(itertools.product(cntTh,lesTh))
Xx = []
Yy = []
gg = 0.0
kk = 0.0
for ii,item in enumerate(combList[:]):
    cnt = pkl.load(open(item[0],'rb'))
    lsn = pkl.load(open(item[1],'rb'))
    if len(lsn) and len(cnt):
        #a,b = np.polyfit(cnt[0],cnt[1],1)
        #c,d = np.polyfit(lsn[0],lsn[1],1)
        cntFit = sm.OLS( cnt[1],cnt[0] ).fit()
        lsnFit = sm.OLS( lsn[1],lsn[0] ).fit()
        x= lsnFit.params
        Xx.append(x)
        xerr = np.abs(x-lsnFit.conf_int())/10
        #xerr_up = [lsnFit.conf_int()[0]-x]
        y = cntFit.params
        Yy.append(y)
        yerr = np.abs(y-cntFit.conf_int())/10
        #yerr_up = [cntFit.conf_int()[0]-y]
        if y>x:
            gg+=1
        else:
            kk+=1
            
    else:
        a=-1
        b=-1
        c=-1
        d= -1
    #print ii
    ax121.scatter(x, y,marker='.') #yerr=list(yerr), xerr=list(xerr),color=clrs[-1],fmt='.', ecolor='g', capthick=0.1)
ax121.plot([0.4,1.6],[0.4,1.6],'k--')
ax121.set_xlim(0.85,1.25)
ax121.set_ylim(0.9,1.45)
ax121.set_xticks([0.9,1.0,1.1,1.2])
ax121.set_yticks([1.0,1.2,1.4])
ax121.set_xlabel('Lesioned')
ax121.set_ylabel('Control')
ax121.set_title('Regression slope')
ax121.xaxis.tick_bottom()
ax121.yaxis.tick_left()

In [89]:
print ks_2samp(cDkde.density,lDkde.density),'\n',ranksums(cDkde.density,lDkde.density),'\n',mannwhitneyu(cDkde.density,lDkde.density)
print '--------------------------------------------------------'
print ks_2samp(cCkde.density,lCkde.density),'\n',ranksums(cCkde.density,lCkde.density),'\n',mannwhitneyu(cCkde.density,lCkde.density)
#pl.plot(lDkde.support,lDkde.density)
#pl.plot(cDkde.support,cDkde.density)

Ks_2sampResult(statistic=0.6171875, pvalue=1.7840354294727415e-86) 
RanksumsResult(statistic=-18.341439670879947, pvalue=3.8642679335870095e-75) 
MannwhitneyuResult(statistic=44281.0, pvalue=1.9357527073600018e-75)
--------------------------------------------------------
Ks_2sampResult(statistic=0.103515625, pvalue=0.0075929452886821175) 
RanksumsResult(statistic=0.41885372247910563, pvalue=0.67532303990092546) 
MannwhitneyuResult(statistic=129090.0, pvalue=0.33770013463007598)


In [90]:
print ranksums(ThprcC[outerC],ThprcL[outerL])
print ks_2samp(ThprcC[outerC],ThprcL[outerL])
print ttest_ind(ThprcC[outerC],ThprcL[outerL])

RanksumsResult(statistic=2.4831867058713519, pvalue=0.013021281888687591)
Ks_2sampResult(statistic=0.42500000000000004, pvalue=0.056992016142655581)
Ttest_indResult(statistic=2.2559612426608329, pvalue=0.030618265501949751)


In [70]:
print ranksums(cntThPost[outerC],lesThPost[outerL])
print ks_2samp(cntThPost[outerC],lesThPost[outerL])
print ttest_ind(cntThPost[outerC],lesThPost[outerL])

RanksumsResult(statistic=1.2097576259373253, pvalue=0.226371910376445)
Ks_2sampResult(statistic=0.33750000000000002, pvalue=0.21186996549631365)
Ttest_indResult(statistic=1.2444097638395852, pvalue=0.22185912330126947)


In [71]:
Xx = np.array(Xx)
Yy = np.array(Yy)


In [72]:
ttest_ind(Xx,Yy),ranksums(Xx,Yy),mannwhitneyu(Xx,Yy)

(Ttest_indResult(statistic=array([-2.46542029]), pvalue=array([ 0.01666574])),
 RanksumsResult(statistic=-1.7741310502909748, pvalue=0.076041475830243357),
 MannwhitneyuResult(statistic=330.0, pvalue=0.038029558070528886))

In [73]:
pl.plot(np.linspace(0,4,100)[:-1],1.0*np.cumsum(np.histogram(cntThPost,bins=np.linspace(0,4,100))[0])/len(cntThPost),'k')
pl.plot(np.linspace(0,4,100)[:-1],1.0*np.cumsum(np.histogram(lesThPost,bins=np.linspace(0,4,100))[0])/len(lesThPost),'r')

In [74]:
data = pd.DataFrame(np.array([cntThPre,cntThPost,['Control']*cntThPost.size]).T,columns=['pre','post','group'])
df = pd.DataFrame(np.array([lesThPre,lesThPost,['Lesion']*lesThPost.size]).T,columns=['pre','post','group'])
data = data.append(df,ignore_index=True)
data[['pre','post']] = data[['pre','post']].astype(float)

In [75]:
from statsmodels.formula.api import ols,rlm
from statsmodels.stats.anova import anova_lm
formula = 'post ~ pre + group-1'  # ANCOVA formula
lm = ols(formula, data)
Rlm = rlm(formula, data)
fit = lm.fit()
Rfit = Rlm.fit()
print fit.summary()
print fit.pvalues
print Rfit.summary()
print Rfit.pvalues

                            OLS Regression Results                            
Dep. Variable:                   post   R-squared:                       0.917
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     236.2
Date:                 Do, 02 Aug 2018   Prob (F-statistic):           6.43e-24
Time:                        11:06:14   Log-Likelihood:                 6.4747
No. Observations:                  46   AIC:                            -6.949
Df Residuals:                      43   BIC:                            -1.463
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
group[Control]    -0.1716      0.097     -1.

In [143]:
interX_lm = ols("post ~ group * pre-1", data).fit()
print interX_lm.summary()
print interX_lm.pvalues

                            OLS Regression Results                            
Dep. Variable:                   post   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                     160.8
Date:                Mi, 22 Mär 2017   Prob (F-statistic):           7.16e-42
Time:                        15:49:34   Log-Likelihood:                 39.954
No. Observations:                 124   AIC:                            -71.91
Df Residuals:                     120   BIC:                            -60.63
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------
group[Control]         -0.3322     

In [20]:
print anova_lm(fit, interX_lm)

NameError: name 'anova_lm' is not defined

In [131]:
lmC = ols("post ~ pre -1", data[data.group=='Control']).fit()
lmL = ols("post ~ pre -1", data[data.group=='Lesion']).fit()

In [146]:
print lmC.params,lmC.pvalues
print lmL.params,lmL.pvalues

pre    1.200688
dtype: float64 pre    8.978122e-44
dtype: float64
pre    1.043664
dtype: float64 pre    6.380734e-69
dtype: float64


In [22]:
GammaDF = pd.DataFrame(zip(lesThPre,lesThPost,['lesion']*lesThPre.size),columns=['A_pre','A_post','group']).append(
          pd.DataFrame(zip(cntThPre,cntThPost,['control']*lesThPre.size),columns=['A_pre','A_post','group']),ignore_index=True)

In [45]:
savemat('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/gammaDF.mat',GammaDF.to_dict(orient='list'))

In [20]:
savemat('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/slopeDF.mat',slopeDB.to_dict(orient='list'))

In [20]:
contourSet = pkl.load(open('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/shuffledGammaContours.pkl','rb'))['contours']
selContourIdx = pkl.load(open('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/shuffledGammaContours.pkl','rb'))['selectedContours']


/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/matplotlib/figure.py:1689: UserWarning: This figure was saved with matplotlib version 1.5.1 and is unlikely to function correctly.
  (version, ))


In [24]:
for idx in selContourIdx[:]:
    #pl.scatter(idx+1,cdfArray[idx+1],c='r',s=50)
    pth = contourSet[idx+1].get_paths()[0]
    pl.plot(pth.vertices[:,0],pth.vertices[:,1],c=seabC[0],lw=1)